In [ ]:
import plotly.express as px
from plotly.offline import iplot
import pandas as pd

# Plotly 
`plotly` is a python package (also and `R` package) that creates interactive graphics with relatively little overhead by the user. Because typically you must input large amounts of data into `plotly` visuals, `pandas` often is used in tandem with `plotly`. The following example imports two data files, filters some data, merges two dataframes, and then feeds needed information into `plotly` to produce line graphs.

### Read in data files

In [ ]:
worldpop = pd.read_csv("data/worldpop.csv")
covid = pd.read_csv("data/covid-data.csv")

In [ ]:
covid

In [ ]:
worldpop

In [ ]:
covid.date = pd.to_datetime(covid.date)
print(covid.date)

In [ ]:
vaccine = covid[covid.date>'2021-01-01']

In [ ]:
vaccine

### Plot time-series chart of vaccines by country

In [ ]:
fig = px.line(vaccine, x = "date", y = "total_vaccinations", color = "location",
              labels = {"total_vaccinations": "Total Vaccinations"},
              title = "COVID-19 Vaccine Doses Administered")
# px.plotly.iplot(fig)
iplot(fig)

### Plot vaccinations per capita by country

#### Data Wrangling

In [ ]:
vaccine_grouping = vaccine.groupby("iso_code").last().reset_index()

In [ ]:
vaccine_grouping

In [ ]:
px.bar(vaccine_grouping.sample(n=10),y='location',x='people_vaccinated',orientation='h')

In [ ]:
merged_df = worldpop.merge(vaccine_grouping, left_on = "Country Code", right_on = "iso_code")[["Country Code", "location", "people_vaccinated","Population"]]

In [ ]:
merged_df

In [ ]:
merged_df["percent_vaccinated"] = merged_df["people_vaccinated"] / merged_df["Population"]

In [ ]:
def change_color(c):
    if c["percent_vaccinated"] < 0.3:
        return "Low Vaccination Rate"
    elif c["percent_vaccinated"] > 0.6:
        return "High Vaccination Rate"
    else:
        return "Moderate Vaccination Rate"

In [ ]:
merged_df["vaccination_rate"] = merged_df.apply(change_color, axis = 1)

#### Plot

In [ ]:
percent_vaccinated = px.bar(merged_df.sample(n=10), x = "location", y = "percent_vaccinated", color = "vaccination_rate",
                            labels = {"location":"Country","percent_vaccinated": "Percent of Pop. Vaccinated", "vaccination_rate": "Vaccination Rate"},
                            title = "COVID-19 Vaccinations Rates")
iplot(percent_vaccinated)